In [1]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import csv
import shutil
import os
import time
import re
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('stopwords')
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import numpy as np

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import pickle
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.cluster import MiniBatchKMeans

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [4]:
from collections import Counter
from random import seed
from random import randrange
from tabulate import tabulate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from scipy.sparse import csr_matrix
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

In [5]:
#Dataset Link: https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt
import requests

base_url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/"
file_name = "amazon_reviews_us_Wireless_v1_00"
r = requests.get(base_url+file_name+".tsv.gz", allow_redirects=True)
#Downlaod data from the server to local machine
open(file_name+".tsv.gz", 'wb').write(r.content)
#Unzip downlaoded data
with gzip.open(file_name+".tsv.gz", 'rb') as f_in:
    with open(file_name+".tsv", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
#Remove old zipped data
os.remove(file_name+".tsv"+".gz")
#Store data into a pandas dataframe
chunk = pd.read_csv(file_name+".tsv", sep='\t',error_bad_lines=False,chunksize=1000000,low_memory=False)
df = pd.concat(chunk)
#Remove unzipped file from local machine
os.remove(file_name+".tsv")
print(len(df))
print(df.head())
print("Before cleanning reviews")
print(df["review_body"].head(5))
def clean_dataset(X):
  review = X
  stemmer = WordNetLemmatizer()

  # Remove all the special characters
  review = re.sub(r'\W', ' ', review)
  
  # remove all single characters
  review = re.sub(r'\s+[a-zA-Z]\s+', ' ', review)
  
  # Remove single characters from the start
  review = re.sub(r'\^[a-zA-Z]\s+', ' ', review) 
  
  # Substituting multiple spaces with single space
  review = re.sub(r'\s+', ' ', review, flags=re.I)
  
  # Removing prefixed 'b'
  review = re.sub(r'^b\s+', '', review)
  
  # Converting to Lowercase
  review = review.lower()
  
  # Lemmatization
  review = review.split()
  review = [stemmer.lemmatize(word) for word in review]
  review = ' '.join(review)

  #Removing all  stopwords.
  review = remove_stopwords(review)
  return review

start_time = time.time()

X = df["review_body"].apply(lambda x: clean_dataset(str(x)))
print(f"Total time to clean reviews: {time.time()-start_time}")
print("After cleanning reviews")
print(X.head(5))
y = df["star_rating"]
from google.colab import drive
drive.mount('/content/drive')
datasetPath = "/content/drive/My Drive/Natural Language Processing/Project/Cleaned Dataset/"
#Saving cleaned data to CSV File
X.to_csv(datasetPath+'X_cleaned.csv', index=False) 
y.to_csv(datasetPath+'y_cleaned.csv', index=False) 

In [6]:
from google.colab import drive
#Reading saved clean file
drive.mount('/content/drive')
datasetPath = "/content/drive/My Drive/Natural Language Processing/Project/Cleaned Dataset/"
start_time = time.time()
X = pd.read_csv(datasetPath+'X_cleaned.csv')
y = pd.read_csv(datasetPath+'y_cleaned.csv')
print(f"Total time to load reviews: {time.time()-start_time}")

#Removing null values
result = pd.concat([X,y], axis=1).dropna()
X = result["review_body"]
y = result["star_rating"]
#Cnverting to 5 ratings 0 - 4 from 1 - 5
def reset_ratings(r):
  if r == 1:
    return 0;
  if r == 2:
    return 0;
  if r == 3:
    return 1;
  if r == 4:
    return 2;
  if r == 5:
    return 2;
y = y.apply(lambda x: reset_ratings(int(x)))
#Calculating Targets or Classes
target_names = []
labels = np.unique(y)
for label in labels:
  target_names.append('Star Rating '+str(label))
print(f"Target Names: {[target_names]}")
#Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train = np.array(X_train).reshape(X_train.shape[0],)
X_test = np.array(X_test).reshape(X_test.shape[0],)
#Convert all rating to type int
y_train = np.array(y_train).astype(int)
y_test = np.array(y_test).astype(int)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Total time to load reviews: 27.96455955505371
Target Names: [['Star Rating 0', 'Star Rating 1', 'Star Rating 2']]
(6013513,)
(6013513,)
(2961880,)
(2961880,)


In [7]:
def transform(array):    
    vectorizer = CountVectorizer(max_features=100, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
    vectorized = vectorizer.fit_transform(array)
    transformer = TfidfTransformer()
    transformed = transformer.fit_transform(vectorized)
    return np.array(transformed.toarray())

In [8]:
start_time = time.time()
print("Creating Count Vectors for training data")
X_train = transform(X_train)
print(np.array(X_train).shape)
print("Creating Count Vectors for testing data")
X_test = transform(X_test)
print(np.array(X_test).shape)
print(f"Total time to vectorize: {time.time() - start_time}")

Creating Count Vectors for training data
(6013513, 100)
Creating Count Vectors for testing data
(2961880, 100)
Total time to vectorize: 250.31044507026672


In [10]:
# convert to sparse matrix to dense matrix
start_time = time.time()
print("Converting sparse matrix to dense matrix")
X_train = csr_matrix(X_train)
X_test = csr_matrix(X_test)
print(f"Total time to vectorize: {time.time() - start_time}")

Converting sparse matrix to dense matrix
Total time to vectorize: 9.82465386390686


In [11]:
# #scaling vector between 0 - 1 
# scaler = MinMaxScaler()
# scaler.fit(X_train)
# X_train =scaler.transform(X_train)

# scaler = MinMaxScaler()
# scaler.fit(X_test)
# X_test = scaler.transform(X_test)

X_train = X_train.toarray()
X_test= X_test.toarray()

In [14]:
from sklearn.linear_model import SGDClassifier
n_batches = 10
current_batch = 0
increment = (int(len(X_train)/n_batches))
clf = SGDClassifier(loss="hinge", penalty="l2")

print(f"Classifying using SVM")
start_time0 = time.time()
for i in range(n_batches):
  start_time = time.time()
  next_batch = current_batch + increment
  clf.partial_fit(X_train[current_batch:next_batch], y_train[current_batch:next_batch], classes=labels)
  current_batch = next_batch
  print(f"Batch {i} : {current_batch} of {len(X_train)} : {time.time() - start_time} seconds ")
print(f"Total training time: {time.time() - start_time0} ")
start_time = time.time()
svm_accuracy = accuracy_score(y_test, clf.predict(X_test))
print(f"Test dataset accuracy for classifier is {svm_accuracy} ")
print(classification_report(y_test, clf.predict(X_test), target_names=target_names))
print(f"Total time to test: {time.time()-start_time}")

Classifying using SVM
Batch 0 : 601351 of 6013513 : 1.3505747318267822 seconds 
Batch 1 : 1202702 of 6013513 : 1.4387307167053223 seconds 
Batch 2 : 1804053 of 6013513 : 1.3935582637786865 seconds 
Batch 3 : 2405404 of 6013513 : 1.2907111644744873 seconds 
Batch 4 : 3006755 of 6013513 : 1.35233736038208 seconds 
Batch 5 : 3608106 of 6013513 : 1.3041882514953613 seconds 
Batch 6 : 4209457 of 6013513 : 1.319347620010376 seconds 
Batch 7 : 4810808 of 6013513 : 1.393507719039917 seconds 
Batch 8 : 5412159 of 6013513 : 1.4104468822479248 seconds 
Batch 9 : 6013510 of 6013513 : 1.3850500583648682 seconds 
Total training time: 13.641655921936035 
Test dataset accuracy for classifier is 0.6721072426972058 


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

Star Rating 0       0.35      0.24      0.29    612674
Star Rating 1       0.00      0.00      0.00    268689
Star Rating 2       0.72      0.89      0.80   2080517

     accuracy                           0.67   2961880
    macro avg       0.36      0.38      0.36   2961880
 weighted avg       0.58      0.67      0.62   2961880

Total time to test: 7.6421709060668945


In [15]:
start_time = time.time()
print(f"Classifying using multinomialNB")
clf = MultinomialNB()
clf.fit(X_train, y_train)
print(f"Total time to train: {time.time()-start_time}")
start_time = time.time()
nb_accuracy = accuracy_score(y_test, clf.predict(X_test))
print(f"Test dataset accuracy is {nb_accuracy} ")
print(classification_report(y_test, clf.predict(X_test), target_names=target_names))
print(f"Total time to test: {time.time()-start_time}")

Classifying using multinomialNB
Total time to train: 3.102656126022339
Test dataset accuracy is 0.6926685078396154 


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

Star Rating 0       0.36      0.07      0.12    612674
Star Rating 1       0.00      0.00      0.00    268689
Star Rating 2       0.71      0.96      0.82   2080517

     accuracy                           0.69   2961880
    macro avg       0.35      0.35      0.31   2961880
 weighted avg       0.57      0.69      0.60   2961880

Total time to test: 7.187659502029419


In [16]:
n_batches = 1
current_batch = 0
increment = (int(len(X_train)/n_batches))
clf = MiniBatchKMeans(n_clusters=len(labels),random_state=0,batch_size=n_batches)

print(f"Classifying using KNN")
start_time0 = time.time()
for i in range(n_batches):
  start_time = time.time()
  next_batch = current_batch + increment
  clf.partial_fit(X_train[current_batch:next_batch])
  current_batch = next_batch
  print(f"Batch {i} : {current_batch} of {len(X_train)} : {time.time() - start_time} seconds ")
print(f"Total training time: {time.time() - start_time0} ")
start_time = time.time()
knn_accuracy = accuracy_score(y_test, clf.predict(X_test))
print(f"Test dataset accuracy for classifier is {knn_accuracy} ")
print(classification_report(y_test, clf.predict(X_test), target_names=target_names))
print(f"Total time to test: {time.time()-start_time}")

Classifying using KNN
Batch 0 : 6013513 of 6013513 : 15.1807279586792 seconds 
Total training time: 15.181747436523438 
Test dataset accuracy for classifier is 0.35008102961632476 
               precision    recall  f1-score   support

Star Rating 0       0.23      0.61      0.33    612674
Star Rating 1       0.07      0.11      0.08    268689
Star Rating 2       0.72      0.30      0.43   2080517

     accuracy                           0.35   2961880
    macro avg       0.34      0.34      0.28   2961880
 weighted avg       0.56      0.35      0.38   2961880

Total time to test: 2103.579432487488
